In [ ]:
!pip install git+https://github.com/vcoscrato/uncertain.git#egg=uncertain

  Cloning https://github.com/vcoscrato/uncertain.git to /tmp/pip-install-__d0rsx8/uncertain
  Running command git clone -q https://github.com/vcoscrato/uncertain.git /tmp/pip-install-__d0rsx8/uncertain
  Created wheel for uncertain: filename=uncertain-0.0.1-cp36-none-any.whl size=23740 sha256=47332510f5d4704862409e6a5e330fd950544bb145f2434e6e2c3e391a51fdbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-v_wctb5y/wheels/8d/54/55/f301254d9c7ee96a6c007b4d92a1e9b8f4cfe6300dcfad2455
Successfully built uncertain


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My\ Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [ ]:
import torch
import pickle
import numpy as np
from copy import deepcopy
from pandas import factorize, read_csv

from uncertain import Interactions
from uncertain.models import FunkSVD, CPMF, OrdRec, MultiModelling
from uncertain.models import LinearUncertainty, CVUncertainty, PlugIn

data = read_csv('Movielens/ratings.csv')
data['movieId'] = factorize(data['movieId'])[0]
data = data.sample(frac=1, random_state=1)
interactions = torch.tensor(data.iloc[:, :2].to_numpy()).cuda()
ratings = torch.tensor(data['rating'].to_numpy()).cuda()

cutoff_train = int(0.6 * len(data))
cutoff_val = cutoff_train + int(0.2 * len(data))

train_idx = slice(None, cutoff_train)
validation_idx = slice(cutoff_train, cutoff_val)
test_idx = slice(cutoff_val, None)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.fit(train, val)

with open('Movielens/results.txt', 'a') as f:
  print('baseline = ', baseline.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

user_support = PlugIn(baseline, LinearUncertainty(None, -train.get_user_support()))

with open('Movielens/results.txt', 'a') as f:
  print('usup = ', user_support.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

item_support = PlugIn(baseline, LinearUncertainty(None, -train.get_item_support()))

with open('Movielens/results.txt', 'a') as f:
  print('isup = ', item_support.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

item_variance = PlugIn(baseline, LinearUncertainty(None, train.get_item_variance()))

with open('Movielens/results.txt', 'a') as f:
  print('ivar = ', item_variance.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 2,
             'batch_size': 512, 'path': 'Movielens/Fitted/funksvdcv.pth', 'use_cuda': True}
funksvdcv = CVUncertainty(recommender=baseline, params=MF_params)
funksvdcv.fit(train, val)
funksvdcv = PlugIn(baseline, funksvdcv.uncertainty)

with open('Movielens/results.txt', 'a') as f:
  print('funksvdcv = ', funksvdcv.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

MF_params = {'embedding_dim': 0, 'l2': 0, 'learning_rate': 2,
             'batch_size': 512, 'path': 'Movielens/Fitted/biascv.pth', 'use_cuda': True}
biascv = CVUncertainty(recommender=baseline, params=MF_params)
biascv.fit(train, val)
biascv = PlugIn(baseline, biascv.uncertainty)

with open('Movielens/results.txt', 'a') as f:
  print('biascv = ', biascv.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

resample = MultiModelling(base_model=baseline, n_models=5, type='Resample')
resample.fit(train, val)

with open('Movielens/results.txt', 'a') as f:
  print('resample = ', resample.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

MF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
             'batch_size': 512, 'path': 'Movielens/Fitted/baseline.pth', 'use_cuda': True}
baseline = FunkSVD(**MF_params)
baseline.initialize(train)
baseline._net.load_state_dict(torch.load(baseline._path))

ensemble = MultiModelling(base_model=baseline, n_models=5, type='Ensemble')
ensemble.fit(train, val)

with open('Movielens/results.txt', 'a') as f:
  print('ensemble = ', ensemble.evaluate(test, train), '\n', file=f)

In [ ]:
train = Interactions(interactions[train_idx], ratings[train_idx])
val = Interactions(interactions[validation_idx], ratings[validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

CPMF_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 4,
               'batch_size': 512, 'path': 'Movielens/Fitted/CPMF.pth', 'use_cuda': True}
cpmf = CPMF(**CPMF_params)
cpmf.fit(train, val)

with open('Movielens/results.txt', 'a') as f:
  print('cpmf = ', cmpf.evaluate(test, train), '\n', file=f)

In [ ]:
factorized_ratings = factorize(ratings.cpu().numpy(), sort=True)
ratings_ = factorized_ratings[0].cuda()
labels = torch.tensor(factorized_ratings[1]).cuda()

train = Interactions(interactions[train_idx], ratings_[train_idx])
val = Interactions(interactions[validation_idx], ratings_[0][validation_idx])
test = Interactions(interactions[test_idx], ratings[test_idx])

OrdRec_params = {'embedding_dim': 50, 'l2': 0, 'learning_rate': 10,
                 'batch_size': 512, 'path': 'Movielens/Fitted/OrdRec.pth', 'use_cuda': True}
ordrec = OrdRec(labels, **OrdRec_params)
ordrec.fit(train, val)

with open('Movielens/results.txt', 'a') as f:
  print('ordrec = ', ordrec.evaluate(test, train), '\n', file=f)